In [54]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import glob

def decay(x,a,b,c):
    #c = tau    
    return a + b*np.exp(-x/c)

data_path = 'ringdown raw data/11-05-2021 633/20211105-0002/'
data_files = glob.glob(data_path + '*.csv')
ring_downs = []
raw_ring_downs = []

for i in data_files[:1]:
    data = pd.read_csv(i, skiprows=1) 
    
    start_full = data.loc[(data['(V)'] == max(data['(V)']))].idxmax()[0]
    end_full = (data['(us)'] >= 350).idxmax()
    #print(start_full, end_full,i)
    data2 = data[start_full:end_full]
    
    
    #need to do 0.9 and 0.1 times the voltage and find the times for those    
    #determine minimum V to get full range
    vrange = max(data2['(V)']) - min(data2['(V)'])
    vmax = max(data2['(V)']) - (0.1 * vrange)
    vmin = min(data2['(V)']) + (0.1 * vrange)
    
    #start = data2.loc[(data2['(V)'] <= vmax)].first_valid_index()
    #end = data2.loc[(data2['(V)'] <= vmin)].first_valid_index()
    #print(start,end)
    start = (data2['(V)'] <= vmax).idxmax()
    end = (data2['(V)'] <= vmin).idxmax()
    endval = data.loc[(start)][1]
    print(endval)
    
    
    time = data['(us)']
    time = time[start_full:end_full]
    voltage = data['(V)']
    voltage = voltage[start_full:end_full]
    
    popt, pcov = curve_fit(decay, time, voltage)
    
    residuals = voltage - decay(time, *popt)
    res_ss = np.sum(residuals**2)
    ss_tot = np.sum((voltage - np.mean(voltage))**2)
    r_squared = 1 - (res_ss / ss_tot)
    
    raw_time = data2['(us)']
    raw_time = raw_time[start:end]
    raw_ringdown = max(raw_time)-min(raw_time)
    
    limit = 0.998
    if r_squared >= limit:
        ring_downs.append(popt[2]*2.198)
        raw_ring_downs.append(raw_ringdown)
    else:
        print('Rejecting poor fit.')
        

print('Minimum R-Squared: ', limit)    
print('Successful ringdown fits: ', len(ring_downs),)
print('Average calculated ringdown time: ', np.mean(ring_downs), ' (us)')
print('Average raw ringdown time: ', np.mean(raw_ring_downs), ' (us)')
print('Calculated ringdown Standard Deviation: ', np.std(ring_downs), ' (us)')
print('Raw ringdown Standard Deviation: ', np.std(raw_ring_downs), ' (us)')
print('min: ',min(raw_ring_downs),'max: ',max(raw_ring_downs))


#Can't reproduce the data generated by the picoscope software exactly, 3us off roughly for some reason and standard deviation also slightly off. 3.19



0.8346457
Minimum R-Squared:  0.998
Successful ringdown fits:  1
Average calculated ringdown time:  47.68302107222689  (us)
Average raw ringdown time:  47.70399865  (us)
Calculated ringdown Standard Deviation:  0.0  (us)
Raw ringdown Standard Deviation:  0.0  (us)
min:  47.70399865 max:  47.70399865
